In [1]:
%pip install dremio-simple-query

Note: you may need to restart the kernel to use updated packages.


In [5]:
from dremio_simple_query.connect import get_token

## URL to Login Endpoint
login_endpoint = "http://172.18.0.5:9047/apiv2/login"

## Payload for Login
## Update with your Dremio credentials

payload = {
  "userName": "dremel",
  "password": "minioadmin1"
}

token = get_token(uri = login_endpoint, payload=payload)

In [6]:
from pyarrow import flight

# Connect to Dremio using Flight
client = flight.FlightClient(f"grpc+tcp://172.18.0.5:32010")

# Authenticate
headers = [
  (b'authorization', f"bearer {token}".encode('utf-8'))
]

# Query to get all product reviews
sql = "SELECT * FROM catalog.silver.vehicle_health_logs"

# Create a Flight descriptor
descriptor = flight.FlightDescriptor.for_command(sql)

# Perform the query using call_options to include headers
options = flight.FlightCallOptions(headers=headers)
flight_info = client.get_flight_info(descriptor, options=options)
reader = client.do_get(flight_info.endpoints[0].ticket, options=options)

# Convert to Pandas DF
vehicle_health_logs = reader.read_pandas()

# Display the dataframe
vehicle_health_logs.head()

,VehicleID,Model,ManufacturingYear,Alerts,MaintenanceHistory
0,VEH20442,AeroFlow,2021,"[{'AlertID': 'ALERT8942', 'Date': '2021-01-25'...","[{'Date': '2023-05-10', 'Details': {'BatteryHe..."
1,VEH23016,AeroFlow,2024,"[{'AlertID': 'ALERT7020', 'Date': '2020-05-08'...","[{'Date': '2022-12-05', 'Details': {'BatteryHe..."
2,VEH67132,AeroFlow,2023,"[{'AlertID': 'ALERT5049', 'Date': '2022-02-06'...","[{'Date': '2023-05-16', 'Details': {'BatteryHe..."
3,VEH31263,EcoSprint,2022,"[{'AlertID': 'ALERT9173', 'Date': '2021-08-25'...","[{'Date': '2024-06-18', 'Details': {'BatteryHe..."
4,VEH62953,UrbanGlide,2022,"[{'AlertID': 'ALERT8582', 'Date': '2020-02-18'...","[{'Date': '2022-11-21', 'Details': {'BatteryHe..."
